# import dataset

In [331]:
import pandas as pd
import numpy as np
from AICScore import AICScore
from pgmpy import estimators
from pgmpy.estimators import BdeuScore, K2Score, BicScore , Aic
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel
from pgmpy.models.BayesianModel import BayesianModel
from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import TabularCPD
import numpy
from pomegranate import BayesianNetwork


df = pd.read_csv('nursery.data', names=['parents', 'has_nurs', 'form', 'children', 'housing', 'finance','social','health','class'])


# learning optimal Bayesian network(gold standard network)

In [138]:
model = BayesianNetwork.from_samples(df, algorithm='exact-dp')
model.structure

((), (), (), (8,), (5, 8), (8,), (8,), (8,), (0, 1))

In [139]:
nursury = BayesianModel([('class', 'children'),('class', 'housing'),('class', 'finance'),('class', 'social'), ('class', 'health'),
                         ('parents', 'class'),('has_nurs', 'class'),('finance','housing')])
nursury.add_node('form')

In [157]:
modele = BayesianNetwork.from_samples(df, algorithm='exact')
modele.structure

((), (), (), (8,), (8,), (4, 8), (8,), (8,), (0, 1))

In [204]:
nursury = BayesianModel([('class', 'children'),('class', 'housing'),('class', 'finance'),('class', 'social'),
                         ('class', 'health'), ('parents', 'class'),('has_nurs', 'class'),('housing','finance')])
nursury.add_node('form')

reverse_gold=BayesianModel([('children', 'class'),('housing', 'class'),('finance', 'class'),('social', 'class'),
                         ('health', 'class'), ('class', 'parents'),('class', 'has_nurs'),('finance','housing')])
reverse_gold.add_node('form')

# parameter estimation of gold standard network

In [247]:
#mle = MaximumLikelihoodEstimator(nursury, df)
nursury.fit(df, estimator=MaximumLikelihoodEstimator)
#cpd_class=mle.estimate_cpd('finance')
for cpd in nursury.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of children:
+----------------+------------------+-----------------+------------------+-------------------+-------------------+
| class          | class(not_recom) | class(priority) | class(recommend) | class(spec_prior) | class(very_recom) |
+----------------+------------------+-----------------+------------------+-------------------+-------------------+
| children(1)    | 0.25             | 0.282700421941  | 1.0              | 0.19881305638     | 0.451219512195    |
+----------------+------------------+-----------------+------------------+-------------------+-------------------+
| children(2)    | 0.25             | 0.255977496484  | 0.0              | 0.239366963403    | 0.30487804878     |
+----------------+------------------+-----------------+------------------+-------------------+-------------------+
| children(3)    | 0.25             | 0.230661040788  | 0.0              | 0.280909990109    | 0.121951219512    |
+----------------+------------------+-----------------+--------

# Sampling from gold standard network

In [248]:
inference = BayesianModelSampling(nursury)
inference.forward_sample(10)

,parents,has_nurs,class,social,health,children,housing,form,finance
0,1,0,0,0,0,2,1,3,0
1,2,0,0,1,0,1,2,0,1
2,2,0,0,1,0,3,0,1,0
3,2,2,1,0,1,1,2,3,0
4,2,3,1,1,2,1,2,0,1
5,2,4,0,0,0,1,0,2,1
6,0,4,3,2,1,3,0,2,0
7,0,4,3,1,2,2,1,2,1
8,2,3,0,1,0,1,2,0,0
9,0,4,0,0,0,2,0,3,0


In [273]:
df['has_nurs'].value_counts()


very_crit      2592
critical       2592
proper         2592
improper       2592
less_proper    2592
Name: has_nurs, dtype: int64

# 1. Sampling from gold standard network
# 2. Learning from the sampled datasets
# 3. Evaluating learned networks

In [291]:
def evals(nursury,reverse_gold,best_model):
    tmp_gold=nursury.copy()
    tmp_best=best_model.copy()
    tmp_rev=reverse_gold.copy()
    n0=tmp_gold.number_of_edges()
    n=tmp_gold.number_of_nodes()
    n2=n*(n-1)/2 #n2=36  :  number of edges in fully connected graph
    
    
    #FP1
    tmp_best.remove_edges_from(nursury)
    FP1=tmp_best.number_of_edges()
    
    #FN1
    tmp_gold.remove_edges_from(best_model)
    FN1=tmp_gold.number_of_edges()
    
    #TP
    TP=n0-FN1
    
    #FP2 , FN2
    tmp_rev.remove_edges_from(best_model)
    FP2=tmp_rev.number_of_edges()
    FN2=tmp_rev.number_of_edges()
    
    #TN
    TN=n2-FN1-FP1-FP2-TP
    
    #FP , FN
    FP = FP1+FP2
    FN = FN1+FN2
    
    #structural evaluation metrics:
    accuracy= (TP+TN)*1./(TP+TN+FP+FN)
    sensitivity= TP*1./(TP+FN)
    AHD=(FP+FN)*1./n
    SHD=FN1+FP1+FP2
    
    print "\t\t",TP,TN,accuracy,sensitivity,AHD,SHD
    

In [338]:
for n_sample in [200,400,600,800,1000,1200,1400,1600,1800,2000,
          3000,4000,5000,6000,7000,8000,9000,10000]:
    inference = BayesianModelSampling(nursury)
    sample=inference.forward_sample(n_sample)
    
    #MDL
    hc = HillClimbSearch(sample, scoring_method=BicScore(sample))
    best_model = hc.estimate()
    print "\nnumer of samples =",n_sample
    print "\tMethod : MDL"
    evals(nursury,reverse_gold,best_model)
    
    #AIC
    hc = HillClimbSearch(sample, scoring_method=AICScore(sample))
    best_model = hc.estimate()
    print "\tMethod : AIC"
    evals(nursury,reverse_gold,best_model)
    
    #BDeu
    for alpha in [0.1, 0.5,10, 1, 5, 10, 20, 50, 80, 100]:
        hc = HillClimbSearch(sample, scoring_method=BdeuScore(sample, equivalent_sample_size=alpha))
        best_model = hc.estimate()
        print "\tMethod : BDeu \talpha =",alpha
        evals(nursury,reverse_gold,best_model)


numer of samples = 200
	Method : MDL
		0 18 0.409090909091 0.0 2.88888888889 18
	Method : AIC
		0 15 0.340909090909 0.0 3.22222222222 21
	Method : BDeu 	alpha = 0.1
		0 19 0.431818181818 0.0 2.77777777778 17
	Method : BDeu 	alpha = 0.5
		0 19 0.431818181818 0.0 2.77777777778 17
	Method : BDeu 	alpha = 10
		0 16 0.363636363636 0.0 3.11111111111 20
	Method : BDeu 	alpha = 1
		0 18 0.409090909091 0.0 2.88888888889 18
	Method : BDeu 	alpha = 5
		0 17 0.386363636364 0.0 3.0 19
	Method : BDeu 	alpha = 10
		0 16 0.363636363636 0.0 3.11111111111 20
	Method : BDeu 	alpha = 20
		0 15 0.340909090909 0.0 3.22222222222 21
	Method : BDeu 	alpha = 50
		0 15 0.340909090909 0.0 3.22222222222 21
	Method : BDeu 	alpha = 80
		0 11 0.25 0.0 3.66666666667 25
	Method : BDeu 	alpha = 100
		0 11 0.25 0.0 3.66666666667 25

numer of samples = 400
	Method : MDL
		0 17 0.386363636364 0.0 3.0 19
	Method : AIC
		0 14 0.318181818182 0.0 3.33333333333 22
	Method : BDeu 	alpha = 0.1
		0 18 0.409090909091 0.0 2.8888888